# Regresión Lineal Ordinaria

# Regresión Lineal Ordinaria
Construcción dataframe dataset
- Cargar archivo
- Borrar columna sin nombre que enumera los ejemplos.
- Dos estructuras, el dataframe y un arreglo que indica si es de entrenamiento.

```python
import pandas as pd; import numpy as np;
datafile = 'prostate.data'; df = pd.read_csv(datafile, sep='\t', header=0);
df = df.drop('Unnamed: 0', axis=1)
istrain_str = df['train']; istrain = np.asarray([True if s == 'T' else False for s in istrain_str])
istest = np.logical_not(istrain)
df = df.drop('train', axis=1)
```

## Descripción del dataset

- Estudio publicado por Tom Stamey (1989, profesor Urología de la Universidad de Stanford)
- Correlación entre nivel de antígeno prostático específico (**lpsa**) y otras mediciones
- Variables
    - lcavol, lweight, age, lbph, svi, lcp, gleason, pgg45
    - Diferentes tipos y escalas
- Dimensiones
    - 97 ejemplos
    - 8 predictores y 1 respuesta (lpsa)

## Normalización

- Es necesaria la normalización de los datos
- Datos en diferentes escalas $\rightarrow$ Distinta influencia posible

```python
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
df_scaled['lpsa'] = df['lpsa']
```

## Regresión Lineal

Se puede obtener la regresión lineal haciendo uso del modulo `linear_model` de sklearn. 

- Necesario agregar una columna `intercept` con unos
- Representa el valor de salida cuando X = 0 para centrar los datos
- Se comporta como una normal estándar apoyando la normalización previamente hecha

```python
import sklearn.linear_model as lm
X = df_scaled.ix[:,:-1]
N = X.shape[0]
X.insert(X.shape[1], 'intercept', np.ones(N))

y = df_scaled['lpsa']
Xtrain = X[istrain]
ytrain = y[istrain]
Xtest = X[np.logical_not(istrain)]
ytest = y[np.logical_not(istrain)]

linreg = lm.LinearRegression(fit_intercept = False)
linreg.fit(Xtrain, ytrain)
```

## Coeficientes y z-scores

Los coeficientes y z-scores asociados a cada predictor pueden apreciarse en la siguiente tabla. Aquellos con valores en negrita corresponden a los coeficientes con suficiente evidencia para demostrar su relación con la respuesta con un nivel de significación de un 5%. 

|            | lcavol | lweight | age    | lbph  | svi   | lcp    | gleason | pgg45 | intercept |
|------------|--------|---------|--------|-------|-------|--------|---------|-------|-----------|
|C           | **0.676**  | **0.262**   | -0.141 | **0.209** | **0.304** | -0.287 | -0.021  | **0.266** | **2.465**     |
|Z           | **5.366**  | **2.751**   | -1.396 | **2.056** | **2.469** | -1.867 | -0.147  | **1.738** | **27.598**|

Los coeficientes se pueden obtener desde `linreg`

El z-score $z_j$ de cada predictor $j$ se pudo obtener con la siguiente formula. 

$$ z_j = \frac{\hat{\beta_j}}{\hat{\sigma}\sqrt{v_j}} $$

Con $X$ siendo la matriz de entrada de dimensiones $N \times (p+1)$ , $y$ siendo el vector de salida, $\hat{y}$ el vector de salida estimado por el modelo, $\hat{\beta_j}$ es el coeficiente asociado al predictor $j$ en el modelo, $\hat{\sigma}$ una estimación la desviación estándar del modelo estimado y $v_j$ un elemento de la matriz $({X^T}X)^{-1}$ usada para construir el modelo lineal.

El valor de $\hat{\sigma}$ es estimado de acuerdo a:

$$ \hat{\sigma}^2 = \frac{1}{N-p-1}\sum_{i=1}^{N}{(y_i - \hat{y}_i)^2} $$


## Validación Cruzada k-fold

Validación Cruzada k-fold divide el conjunto de entrenamiento en k-folds. En cada iteración usa k-1 folds para construir el modelo y el fold restante para validarlo. 

Se construyo el modelo y se comparó una estimación del error del modelo usando validación cruzada con el modelo completo de regresión lineal.


|                     | 10-Fold CV | 5-Fold CV | Linear Regression |
|---------------------|------------|-----------|-------------------|
| MSE promedio (val)  | 0.757      | 0.957     |                   |
| MSE promedio (test) | 0.538      | 0.588     |                   |
| MSE minimo (val)    | 0.136      | 0.414     |                   |
| MSE minimo (test)   | 0.508      | 0.505     |                   |
| MSE (train)         |            |           | 0.567             |
| MSE (test)          |            |           | 0.66              |

## Hipótesis de normalidad

Al observar una Q-Q plot de los residuos se puede observar que los residuos están aproximadamente sobre la línea por lo que es razonable la hipotesis de normalidad.

![alt text](qqplot.png "Q-Q plot")

## Conclusiones

- Se construyó un modelo de regresión lineal usando el modulo `linear_model` de sklearn.

- Rredictores con mayor influencia sobre la respuesta fueron `lcavol`, `lweight` y `svi`.

- Predictores `age`, `gleason` y `lcp` no tenían suficiente evidencia como para tener influencia sobre la respuesta.

- Cross-Validation mostró que el modelo podría tener un cierto sobre-ajuste pues al usar menos datos se mejora el error respecto del conjunto de pruebas.

- Usando un Q-Q plot sobre los residuos se puede concluir que la hipótesis de normalidad es razonable.
